In [ ]:
import logging.config
import os
import pathlib

Set visible gpu.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Configure logging

In [ ]:
output_dir = pathlib.Path('../tune_results')
try:
    os.makedirs(output_dir)
except FileExistsError:
    if not os.path.isdir(output_dir):
        raise FileExistsError("Please provide a path to a non-existing or directory as the output directory.")
log_format = '%(asctime)s: %(levelname)s: %(name)s: %(filename)s: %(funcName)s(): %(lineno)d:\t%(message)s'
formatter = logging.Formatter(log_format)
file_handler = logging.FileHandler(str(output_dir / 'log.log'), mode = 'a')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
formatter = logging.Formatter(fmt="[%(levelname)s %(asctime)s] %(name)s: %(message)s", datefmt="%m-%d %H:%M:%S",)
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
stream_handler.setFormatter(formatter)
logging.basicConfig(level = logging.DEBUG, handlers = [file_handler, stream_handler])

In [ ]:
import ray
import ax.service.ax_client as ax_client
import ray.tune as tune
import ray.tune.schedulers as schedulers
import ray.tune.suggest as suggests
import ray.tune.suggest.ax as ax_suggest

from configs import hyper_parameter_constraints, hyper_parameters_n as hyper_parameters
import tune_hyperparameters

ax_client.logger.addHandler(file_handler)
tune_hyperparameters.seeds = [26360154, 83182821, 39315480, 47450099, 87494286]

Initialize ray

In [ ]:
ray.init()

Configure trail scheduler.

AsyncHyperBandScheduler is recommended over vanilla HyperBand by ray tune official.

In [ ]:
scheduler = schedulers.AsyncHyperBandScheduler(metric = 'mean_accuracy', mode = "max", max_t = 6,
                                               grace_period = 4, reduction_factor = 2)

Use Ax's search algorithm.

It is recommended to not use Repeater with a TrialScheduler.
Early termination can negatively affect the average reported metric.

Set enforce_sequential_optimization to True because there are that many GPU avaliable.

In [ ]:
client = ax_client.AxClient(enforce_sequential_optimization = True)
client.create_experiment(
        parameters = hyper_parameters,
        name = "Ax Experiment",
        objective_name = "best_accuracy",
        parameter_constraints = hyper_parameter_constraints)
search_alg = ax_suggest.AxSearch(client, mode = "max")
# search_alg = suggests.Repeater(search_alg, repeat = 2)
search_alg = suggests.ConcurrencyLimiter(search_alg, max_concurrent = 1)

Setup experement.

In [ ]:
experiment = tune.Experiment(
        name = "My Experiment",
        run = tune_hyperparameters.Trainable,
        stop = {'mean_accuracy': 0.95},
        config = {
                'epoch_per_iteration': 4, 'batch_size': 14,
                'test_room': 2,
                'data_dir': pathlib.Path('/data/wzy/WifiEnvironment/gram_data'),
                'log_format': log_format},
        resources_per_trial = {
                'cpu': 1,
                'gpu': 1},
        num_samples = 70,
        local_dir = output_dir,
        checkpoint_freq = 0,
        checkpoint_at_end = True,
        max_failures = 5)

Run trails.

In [ ]:
analysis = tune.run(
        experiment,
        scheduler = scheduler,
        search_alg = search_alg,
        raise_on_failed_trial = False)

Analyze results.

In [ ]:
logging.getLogger('main').critical("Best config is {}".format(analysis.get_best_config(metric = "best_accuracy")))
trials = analysis.trials
best_trail = analysis.get_best_trial(metric = "best_accuracy")
best_parameters, values = client.get_best_parameters()
logging.getLogger('main').critical("Best config is {}, {}".format(best_parameters, values))

In [ ]:
ray.shutdown()